In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ## next time, load it into your work folder:
# ## dont forget to restart the runtime, so it forgets about the old version !
!cp "/content/drive/My Drive/NASA/cv2_cuda_test/cv2.cpython-310-x86_64-linux-gnu.so" .

%cd /content/drive/MyDrive/NASA

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1AM28lxL021WtA9_4-YXcChNOcijk4IBJ/NASA


In [ ]:
import cv2
count = cv2.cuda.getCudaEnabledDeviceCount()
print(count)

1


In [ ]:
!pip install azure-storage-blob azure-identity --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 9.1 MB/s eta 0:00:00


In [ ]:
from azure.storage.blob import BlobServiceClient
import os
from matplotlib import pyplot as plt
from io import BytesIO
import numpy as np
from numpy import inf
import pandas as pd
import datetime
import math
from math import radians, cos, sin, asin, sqrt

from mpl_toolkits import mplot3d
import numpy as np
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

import io
import random
import time
import torch
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
from PIL import Image, ImageEnhance, ImageOps
from google.colab import userdata
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle

from torchvision import transforms
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pad_packed_sequence, pack_padded_sequence, PackedSequence
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
connection_string = "Add connection string"

# Setup to load file from blob
blob_service_client = "Create Blob Service Client"
container_client = "Create Container Client"

#### Data Ingestion

In [ ]:
# aircraft_metadata_params = ['DateTime_UTC', 'GPS_MSL_Alt', 'Drift', 'Pitch', 'Roll', 'Vert_Velocity']

aircraft_metadata_params = [
    'DateTime_UTC', 'Lat', 'Lon', 'GPS_MSL_Alt', 'WGS_84_Alt', 'Press_Alt',
    'Grnd_Spd', 'True_Airspeed', 'Mach_Number', 'Vert_Velocity', 'True_Hdg',
    'Track', 'Drift', 'Pitch', 'Roll', 'Ambient_Temp', 'Total_Temp',
    'Static_Press', 'Dynamic_Press', 'Cabin_Pressure', 'Wind_Speed',
    'Wind_Dir', 'Solar_Zenith', 'Sun_Elev_AC', 'Sun_Az_Grd', 'Sun_Az_AC'
]

CTH_col = 'top_height'

# Aircraft Metadata
def load_metadata(blob_name):
    blob_client = container_client.get_blob_client(blob_name)
    streamdownloader = blob_client.download_blob()
    metadata_df = pd.read_csv(io.BytesIO(streamdownloader.readall()))
    return metadata_df

# LiDAR Validation Heights
def load_validation_heights(blob_name):
    blob_client = container_client.get_blob_client(blob_name)
    streamdownloader = blob_client.download_blob()
    validation_df = pd.read_csv(io.BytesIO(streamdownloader.readall()))
    return validation_df

##### Data Class

In [ ]:
# CloudDataset classes integrating all 3 data sources: FEGS Images, Aircraft Metadata and LiDAR Validation Heights with temporal alignment
class CloudDataset(Dataset):
    def __init__(self, date_folders, transform=None, drop_after_last_validation=True):
        self.date_folders = date_folders
        self.transform = transform
        self.drop_after_last_validation = drop_after_last_validation
        self.data_df = self._prepare_dataframe()

    def _prepare_dataframe(self):
        """
        Iterates over the date folders in azure blob storage and loads:
          1. .jpg Images from each sub-directory in the folder with '_crop_corrected_aligned' in the name.
          2. Aircraft Metadata with 1-1 time alignment with the images.
          3. LiDAR Validation Heights, mapped using timestamp, if not available filled with NaN.
        Creates a df with following columns:
            timestamp, image_path, [...aircraft_metadata_params...], validation_height
        """
        image_paths, timestamps, metadata_rows, validation_heights = [], [], [], []

        for folder in self.date_folders:
            print(f"Processing folder: {folder}")
            folder_image_paths, folder_timestamps, folder_metadata_rows, folder_validation_heights = [], [], [], []

            blob_list = container_client.list_blobs(name_starts_with=folder)
            metadata_path, validation_path = None, None
            for blob in blob_list:
                # extract image paths of all .jpg images in cropped folders
                # if blob.name.endswith(".jpg") and "_crop_corrected_aligned" in blob.name:
                if blob.name.endswith(".jpg") and "_frames" in blob.name in blob.name:
                    folder_image_paths.append(blob.name)
                    folder_timestamps.append(self._extract_timestamp_from_filename(blob.name))
                # extract the aircraft metadata file path
                if blob.name.startswith(f"{folder}/IWG1.") and "processed" in blob.name:
                    metadata_path = blob.name
                # extract the LiDAR validation file path
                if blob.name.startswith(f"{folder}/goesrplt_CPL_layers_") and blob.name.endswith("_processed.txt"):
                    validation_path = blob.name

            # load aircraft metadata and LiDAR validation data
            if metadata_path:
                metadata_df = load_metadata(metadata_path)
            if validation_path:
                validation_df = load_validation_heights(validation_path)

            # prepare LiDAR validation data
            validation_df['datetime_combined'] = validation_df['date'] + ' ' + validation_df['timestamp']
            validation_df['datetime_combined'] = validation_df['datetime_combined'].str.split('.').str[0]
            validation_df['datetime_combined'] = pd.to_datetime(validation_df['datetime_combined'], format="%Y-%m-%d %H:%M:%S")

            # prepare aircraft metadata
            metadata_df = metadata_df[aircraft_metadata_params]
            metadata_df['DateTime_UTC'] = metadata_df['DateTime_UTC'].str.split('.').str[0]
            metadata_df = self._extract_time_features(metadata_df)  # Add hour_of_day and day_of_year

            metadata_timestamps = pd.to_datetime(metadata_df['DateTime_UTC'], format="%Y-%m-%d %H:%M:%S")
            metadata_df = metadata_df.set_index(metadata_timestamps)
            aligned_metadata = pd.DataFrame(index=pd.to_datetime(folder_timestamps, format="%H:%M:%S"))
            aligned_metadata = aligned_metadata.join(metadata_df, how='left')
            aligned_metadata = aligned_metadata[aircraft_metadata_params + ['hour_of_day', 'day_of_year']]

            folder_metadata_rows.extend(aligned_metadata.values.tolist())

            # extract LiDAR validation height exactly matching the timestamp where available, else NaN
            for ts in folder_timestamps:
                cth = self._map_timestamp_to_lidar(ts, validation_df)
                folder_validation_heights.append(cth)

            # Create a folder-level DataFrame
            folder_data = {
                'timestamp': folder_timestamps,
                'image_path': folder_image_paths,
                **{param: [row[i] for row in folder_metadata_rows] for i, param in enumerate(aircraft_metadata_params + ['hour_of_day', 'day_of_year'])},
                'validation_height': folder_validation_heights
            }
            folder_df = pd.DataFrame(folder_data)

            # Conditionally remove rows after the last valid validation_height in this folder
            if self.drop_after_last_validation:
                last_valid_index = folder_df['validation_height'].last_valid_index()
                if last_valid_index is not None:
                    folder_df_cleaned = folder_df.loc[:last_valid_index].copy()  # Use .copy() to ensure independence
                else:
                    folder_df_cleaned = folder_df.copy()  # In case there are no valid entries
            else:
                folder_df_cleaned = folder_df  # Keep all rows if dropping is disabled

            # Extend to the global lists
            image_paths.extend(folder_df_cleaned['image_path'].tolist())
            timestamps.extend(folder_df_cleaned['timestamp'].tolist())
            metadata_rows.extend(folder_df_cleaned[aircraft_metadata_params + ['hour_of_day', 'day_of_year']].values.tolist())
            validation_heights.extend(folder_df_cleaned['validation_height'].tolist())

            # Print the lengths for the current folder
            print(f"Folder {folder}:")
            print(f"  Number of images: {len(folder_df_cleaned['image_path'])}")
            print(f"  Number of timestamps: {len(folder_df_cleaned['timestamp'])}")
            print(f"  Number of metadata rows: {len(folder_df_cleaned)}")
            print(f"  Number of validation heights: {len(folder_df_cleaned['validation_height'])}")

        # Print the final lengths after processing all folders
        print("After processing all folders combined:")
        print(f"  Total number of images: {len(image_paths)}")
        print(f"  Total number of timestamps: {len(timestamps)}")
        print(f"  Total number of metadata rows: {len(metadata_rows)}")
        print(f"  Total number of validation heights: {len(validation_heights)}")

        # Check for any mismatches
        if not (len(image_paths) == len(timestamps) == len(metadata_rows) == len(validation_heights)):
            print("Error: Length mismatch detected!")
            print(f"  Images: {len(image_paths)}")
            print(f"  Timestamps: {len(timestamps)}")
            print(f"  Metadata rows: {len(metadata_rows)}")
            print(f"  Validation heights: {len(validation_heights)}")
            return None

        # combine all aligned data in a df
        data = {
            'timestamp': timestamps,
            'image_path': image_paths,
            **{param: [row[i] for row in metadata_rows] for i, param in enumerate(aircraft_metadata_params + ['hour_of_day', 'day_of_year'])},
            'validation_height': validation_heights
        }
        df = pd.DataFrame(data)
        df = df.drop(columns=['DateTime_UTC'])

        # Print columns with NaN values before interpolation
        self._print_columns_with_nan(df)

        # Interpolate missing values, excluding 'validation_height'
        df = self._interpolate_missing_values(df)

        # Add sequence length information for RNN
        self._add_sequence_length_column(df)
        return df

    def _extract_time_features(self, df):
        """
        Extracts hour of day and day of year from the DateTime_UTC column.
        Adds 'hour_of_day' (with fractional hour) and 'day_of_year' as new columns in the DataFrame.
        """
        df['DateTime_UTC'] = pd.to_datetime(df['DateTime_UTC'], format="%Y-%m-%d %H:%M:%S")
        df['hour_of_day'] = df['DateTime_UTC'].dt.hour + df['DateTime_UTC'].dt.minute / 60
        df['day_of_year'] = df['DateTime_UTC'].dt.dayofyear
        return df

    def _extract_timestamp_from_filename(self, filename):
        """
        Extracts the timestamp from the image filename on the blob.
        path/to/blob/YYYYMMDD_HHMMSS_frame_n_cropped.jpg -> %Y%m%d%H%M%S
        """
        filename = os.path.basename(filename)
        date_str = filename.split("_")[0]
        time_str = filename.split("_")[1]
        timestamp = datetime.strptime(date_str + time_str, "%Y%m%d%H%M%S")
        return timestamp


    def _map_timestamp_to_lidar(self, timestamp, validation_df):
        """
        extract LiDAR validation height exactly matching the timestamp where available, else NaN
        """
        validation_df['datetime_combined'] = pd.to_datetime(validation_df['datetime_combined'], format="%Y-%m-%d %H:%M:%S")
        timestamp_dt = pd.to_datetime(timestamp, format="%Y-%m-%d %H:%M:%S")
        exact_match = validation_df[validation_df['datetime_combined'] == timestamp_dt]
        return exact_match[CTH_col].values[0] if not exact_match.empty else np.nan


    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        """
        Retrieve a data record from the dataset for a given index.
        Returns loaded and transformed image, metadata and validation height associated with that image if available.
        """
        row = self.data_df.iloc[idx]

        image_path = row['image_path']
        blob_client = container_client.get_blob_client(image_path)
        streamdownloader = blob_client.download_blob()
        img_data = streamdownloader.readall()
        img = Image.open(io.BytesIO(img_data)).convert("RGB")
        if self.transform:
            img = self.transform(img)

        metadata = torch.tensor([row[param] for param in aircraft_metadata_params], dtype=torch.float32)
        validation_height = torch.tensor([row['validation_height']], dtype=torch.float32)

        return img, metadata, validation_height

    def _add_sequence_length_column(self, df):
        # Initialize a new column to NaN
        df['sequence_length'] = np.nan

        # Track the start of each sequence
        sequence_start = 0

        # Iterate through the dataframe to detect when a validation_height exists
        for i in range(len(df)):
            if not pd.isna(df.loc[i, 'validation_height']):
                # We found the end of a sequence, so mark the previous sequence images
                sequence_length = i - sequence_start
                df.loc[sequence_start:i, 'sequence_length'] = sequence_length + 1  # Using count (1-based index)
                sequence_start = i + 1  # Move the start to the next sequence

        # Ensure all sequence_length values are integers (if any were missed)
        df['sequence_length'] = df['sequence_length'].astype(int)

        return df

    def _print_columns_with_nan(self, df):
        """
        Prints the names of columns in the DataFrame that contain NaN values.
        """
        columns_with_nan = df.columns[df.isna().any()].tolist()
        if columns_with_nan:
            print("Columns with NaN values:")
            for col in columns_with_nan:
                print(col)
        else:
            print("No columns with NaN values.")

    def _interpolate_missing_values(self, df, exclude_columns=['validation_height', 'timestamp', 'image_path']):
        """
        Interpolates missing values in the DataFrame for all columns except specified ones.
        """
        # Create a copy to avoid modifying the original DataFrame
        df = df.copy()

        # Store excluded columns
        excluded_data = {col: df[col].copy() for col in exclude_columns if col in df.columns}

        # Convert all object-type columns in df to numeric where possible
        df = df.infer_objects()

        # Get columns for interpolation (excluding specified columns)
        columns_to_interpolate = [col for col in df.columns if col not in exclude_columns]

        # Convert columns to numeric where possible
        for col in columns_to_interpolate:
            try:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            except (ValueError, TypeError):
                continue

        # Perform interpolation only on numeric columns
        numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
        if not numeric_columns.empty:
            df[numeric_columns] = df[numeric_columns].interpolate(
                method='linear',
                axis=0,
                limit_direction='both'
            )

        # Restore excluded columns
        for col, data in excluded_data.items():
            df[col] = data

        return df

##### Insantiating the data class, loading the data and saving the frame

In [ ]:
# train_dates = ["60fps_v1/20170418"]

# # Change the cache_cloud_dataset variable at the start to make this either
# # generate the CloudDataset (takes some time) or use a cached version on Google Drive.

# full_dataset = CloudDataset(train_dates, transform=None)
# # Extract the full dataframe from CloudDataset
# full_dataframe = full_dataset.data_df
# # Ensure folder exists
# os.makedirs('yash_datasets', exist_ok=True)
# # Save the dataframe directly to Google Drive
# full_dataframe.to_csv('yash_datasets/train_dataset.csv', index=False, header=True)

In [ ]:
full_dataframe = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_dataset.csv")

In [ ]:
inputs = full_dataframe.drop_duplicates('timestamp')

In [ ]:
inputs.dropna(subset=['validation_height'], inplace=True)
inputs.head()

,timestamp,image_path,Lat,Lon,GPS_MSL_Alt,WGS_84_Alt,Press_Alt,Grnd_Spd,True_Airspeed,Mach_Number,...,Wind_Speed,Wind_Dir,Solar_Zenith,Sun_Elev_AC,Sun_Az_Grd,Sun_Az_AC,hour_of_day,day_of_year,validation_height,sequence_length
180,2017-04-18 17:57:09,60fps_v1/20170418/170418_175706_183328_frames/...,34.610995,-86.583988,19969.6,19969.8,65045.0,202.8,201.0,0.692,...,3.7,236.4,23.7,67.3,-173.6,-172.5,17.95,108,13161.0,181
480,2017-04-18 17:57:14,60fps_v1/20170418/170418_175706_183328_frames/...,34.620100,-86.583985,19968.1,19968.3,65032.5,202.8,201.0,0.692,...,3.5,236.3,23.7,67.4,-173.6,-172.4,17.95,108,13131.0,241
780,2017-04-18 17:57:19,60fps_v1/20170418/170418_175706_183328_frames/...,34.629211,-86.583980,19967.6,19967.6,65032.5,202.8,201.0,0.692,...,3.6,237.2,23.7,67.4,-173.5,-172.5,17.95,108,13161.0,241
1080,2017-04-18 17:57:24,60fps_v1/20170418/170418_175706_183328_frames/...,34.638317,-86.583978,19967.1,19967.2,65035.0,202.8,200.2,0.691,...,4.0,231.4,23.7,67.4,-173.5,-172.5,17.95,108,13191.0,241
1380,2017-04-18 17:57:29,60fps_v1/20170418/170418_175706_183328_frames/...,34.647425,-86.583978,19966.6,19966.6,65027.5,202.8,200.8,0.692,...,3.9,231.8,23.7,67.4,-173.4,-172.5,17.95,108,13161.0,241


#### Helper functions for image loading, fisheye correction and cropping

In [ ]:
def load_image_from_blob_cv(blob_img, container_client):
    """
    Loads the image from Azure Blob Storage using OpenCV and returns it as a numpy array.
    Args:
        blob_img (str): name of the blob image in the container
        container_client (azure.storage.blob.BlobContainerClient): container client
    Returns:
        (numpy.ndarray): loaded image with original channels retained
    """
    blob_client = container_client.get_blob_client(blob_img)
    streamdownloader = blob_client.download_blob()
    blob_data = streamdownloader.readall()
    image_array = np.asarray(bytearray(blob_data), dtype=np.uint8)
    # img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    img_bgr = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    return img_rgb

def undistort_fisheye_image(distorted_image):
    """
    Apply correction for fisheye distortion.
    Args:
        distorted_image (numpy.ndarray): original image that has the fisheye distortion.
     Returns:
        (numpy.ndarray): undistorted image with fisheye correction.
    """
    # Parameters provided
    f = 1.4  # focal length [mm]
    mu = 2.8e-3  # pixel pitch [mm]
    S = 2  # output (undistorted) image scale factor
    # distortion polynomial order:  [2 4 6 8]
    # polynomial coefficients:
    coeffs = np.array([0.01166363, -0.04819808, 0.07918044, -0.037572])

    H = distorted_image.shape[0]  # image height [pixel]
    W = distorted_image.shape[1]  # image width [pixel]
    cx = (W - 1) / 2  # image center coordinate [pixel]
    cy = (H - 1) / 2  # image center coordinate [pixel]

    K = np.array([[f / mu, 0, cx], [0, f / mu, cy], [0, 0, 1]])

    # compute intrinsic matrix for undistorted image
    cpx = (W * S - 1) / 2
    cpy = (H * S - 1) / 2
    P = np.array([[f / mu, 0, cpx], [0, f / mu, cpy], [0, 0, 1]])

    # rectification matrix (identity)
    R = np.eye(3)

    # produce undistorted image
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K=K, D=coeffs, R=R, P=P, size=[W * S, H * S], m1type=cv2.CV_16SC2)
    undistorted_image = cv2.remap(distorted_image, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_TRANSPARENT)
    return undistorted_image

def crop_and_correct_image_cv2(image, size1=(1500, 1500), offset=(85, 180), size2=(800, 800)):
    """
    Crops the undistorted image in two stages and ensures the original center (960, 540)
    aligns with the center of the final cropped image (400, 400).

    Args:
        image (numpy.ndarray): Input image to crop.
        size1 (tuple): Size of the first crop (width, height).
        offset (tuple): Offset for the first crop.
        size2 (tuple): Size of the final crop (width, height).

    Returns:
        numpy.ndarray: Cropped image.
    """
    h, w = image.shape[:2]

    # Step 1: First crop with offset
    center_h, center_w = h // 2, w // 2  # Center of the undistorted image
    offset_h, offset_w = offset

    # Adjust the starting coordinates for the first crop
    start_h1 = max(center_h - size1[1] // 2 + offset_h, 0)
    start_w1 = max(center_w - size1[0] // 2 + offset_w, 0)
    cropped_image = image[start_h1:start_h1 + size1[1], start_w1:start_w1 + size1[0]]

    # Step 2: Adjust second crop to ensure the original center aligns with the center of the crop-corrected image
    crop_h, crop_w = size2
    center_h_crop = center_h - start_h1  # Adjusted center in the cropped image
    center_w_crop = center_w - start_w1

    # Calculate start coordinates to place the center at 400, 400
    start_h2 = max(center_h_crop - crop_h // 2, 0)
    start_w2 = max(center_w_crop - crop_w // 2, 0)

    final_image = cropped_image[start_h2:start_h2 + crop_h, start_w2:start_w2 + crop_w]

    return final_image

#### Optical flow related functions

In [ ]:
def sampling_framerate(desired_sampling_rate_fps=20, actual_framerate_fps=60):
    """
    For chandrakant's method, minimum desired sampling rate is 2 fps. His original successful run tries 20fps
    :param desired_sampling_rate_fps:
    :param actual_framerate_fps:
    :return:
    """
    fincrement = int(actual_framerate_fps/desired_sampling_rate_fps)
    return fincrement

In [ ]:
def getData(x,y,z):
  temp = z[x,y]
  return temp

f=np.vectorize(getData)

In [ ]:
def get_undistorted_endpoints_1(mu, S, H, W, x, y):
    cx = (W-1)/2        # image center coordinate [pixel]
    cy = (H-1)/2        # image center coordinate [pixel]

    K = np.array([[f/mu,0,cx],[0,f/mu,cy],[0,0,1]])

    # compute intrinsic matrix for ouput image
    cpx = (W*S-1)/2
    cpy = (H*S-1)/2
    P = np.array([[f/mu,0,cpx],[0,f/mu,cpy],[0,0,1]])

    D = np.array([0.01166363, -0.04819808, 0.07918044, -0.037572])

    R = np.eye(3)

    coord_homog = np.array([x, y, 1.0], dtype=np.float32).reshape(-1, 1)
    undistorted_coord = cv2.fisheye.undistortPoints(
        coord_homog.T[:, :2].reshape(1, -1, 2),
        K=K, D=D, R=R, P=P
    )[0][0]
    x_1, y_1 = undistorted_coord

    return x_1, y_1

In [ ]:
def get_speed(first_pixel, last_pixel, first_frame, last_frame):
    mu = 7.7/1082
    travelled_pixels = last_pixel - first_pixel
    travel_time = (last_frame - first_frame)/60

    travelled_distance = travelled_pixels * mu
    #print(travelled_pixels, travelled_distance, travel_time)

    #if travel_time != 0:
    u = travelled_distance/travel_time
    #else:
        #u = 0

    return u

s = np.vectorize(get_speed)

In [ ]:
def Create_Output(X_Coord,Y_Coord,height):
    output = [X_Coord, Y_Coord, height]

    return output

final = np.vectorize(Create_Output)

In [ ]:
def get_center_region(img1_1, r_h, r_w):
  h, w = img1_1.shape

  center_x, center_y = w // 2, h // 2
  half_width, half_height =  r_w // 2, r_h // 2
  center_region = (slice(center_y - half_height, center_y + half_height), slice(center_x - half_width, center_x + half_width))

  return center_region

####Constants

In [ ]:
cnt = 0
maxtrack_time = 5
#last framenum to check where pixel went
end_frame_loop = maxtrack_time * 60
ACTUAL_FRAMERATE = 60 #FPS
H = 600
W = 600

frame_pixels = 1082
mu = 8/1082
f = 1.4
mu_1 = 2.8e-3
S = 2

In [ ]:
inputs = pd.read_excel("/content/drive/MyDrive/Input_1.xlsx")

In [ ]:
#!unzip '/content/drive/MyDrive/frame331.zip'

In [ ]:
img1_1 = load_image_from_blob_cv("170418_175706_183328_All_Frames/frame0.jpg", container_client)
img1 = cv2.cvtColor(img1_1, cv2.COLOR_RGB2GRAY)

center_region = get_center_region(img1, 600, 600)
center_region

(slice(240, 840, None), slice(660, 1260, None))

In [ ]:
fincrement = sampling_framerate(desired_sampling_rate_fps=20, actual_framerate_fps=ACTUAL_FRAMERATE)
fincrement

3

In [ ]:
# Precalculate fisheye corrected co-ordinates
x_corr = np.zeros([H, W])
y_corr = np.zeros([H, W])

for i in range(H):
    for j in range(W):
        k_1, k = get_undistorted_endpoints_1(mu_1, S, H, W, j, i)

        x_corr[i,j] = k
        y_corr[i,j] = k_1

In [ ]:
optical_flow_cuda = cv2.cuda.OpticalFlowDual_TVL1_create()

In [ ]:
for ind in tqdm(inputs.index):
    st = time.time()

    if ind == 2:
      # Get all motion comparisons
      first_frame = inputs['First Frame'][ind]
      results = []
      cnt = 0

      for i in range(first_frame, first_frame+end_frame_loop, fincrement):
        #print(datetime.now(), cnt)
        img1_1 = load_image_from_blob_cv("170418_175706_183328_All_Frames/frame%d.jpg" % i, container_client)
        #img1_1 = cv2.imread('frame%d.jpg' % i, cv2.IMREAD_GRAYSCALE)
        #img1_1 = load_image_from_blob_cv(full_dataframe['image_path'][first_frame], container_client)
        img1_1 = cv2.cvtColor(img1_1, cv2.COLOR_RGB2GRAY)
        img1_1[img1_1<75]=0
        img1 = img1_1[center_region]

        for j in range(i+fincrement, i+31,fincrement):
          img2_1 = load_image_from_blob_cv("170418_175706_183328_All_Frames/frame%d.jpg" % j, container_client)
          #img2_1 = cv2.imread('frame%d.jpg' % j, cv2.IMREAD_GRAYSCALE)
          #img2_1 = load_image_from_blob_cv(full_dataframe['image_path'][j], container_client)
          img2_1 = cv2.cvtColor(img2_1, cv2.COLOR_RGB2GRAY)
          img2_1[img2_1<75]=0
          img2 = img2_1[center_region]

          cuMat1 = cv2.cuda_GpuMat()
          cuMat2 = cv2.cuda_GpuMat()
          cuMat1.upload(img1)
          cuMat2.upload(img2)

          cu_flow = optical_flow_cuda.calc(cuMat1, cuMat2, None)
          optical_flow_data = cu_flow.download()

          results.append([i, j, optical_flow_data])
          cnt += 1

      # Calculate first position change for all frames
      one_cycle = int(30/fincrement)
      total_comparisons = int(end_frame_loop/fincrement * one_cycle)

      movement_all = []
      for frame in range(0, total_comparisons, one_cycle):
          posn_change = np.zeros([H,W])
          v = np.zeros([H, W])
          u = np.zeros([H, W])
          for i in range(one_cycle):
              #print(results[frame+i][0], results[frame+i][1])

              v_1 = np.copy(results[frame+i][2][:,:,0])
              u_1 = np.copy(results[frame+i][2][:,:,1])

              temp1 = (abs(u_1) >=1).astype(int)
              temp2 = (abs(v_1) >=1).astype(int)
              temp3 = temp1 + temp2

              temp4 = (temp3 >=1).astype(int)
              temp5 = (posn_change < 100).astype(int)

              temp6 = np.multiply(temp4, temp5)

              posn_change = posn_change + temp6*(i+1)*100
              u = u + np.multiply(temp6, u_1)
              v = v + np.multiply(temp6, v_1)

          movement_all.append([results[frame][0], posn_change, u, v])

          #with open(str(results[frame][0]) + ".pkl", "wb") as fp:   #Pickling
              #pickle.dump(movement, fp)

          #fp.close()

      #  Calculate path of each pixel
      #with open(str(first_frame) + ".pkl", 'rb') as fp:
        #movement = pickle.load(fp) # deserialize using load()

      #os.remove(str(first_frame) + ".pkl")
      movement = movement_all[0]
      posn = movement[1]
      u = movement[2]
      v = movement[3]

      x = np.arange(H)
      y = np.arange(W)

      x_1, y_1 = np.meshgrid(y,x)

      u_1 = getData(y_1, x_1, u)
      v_1 = getData(y_1, x_1, v)

      x_1 = x_1 + np.round(v_1)
      y_1 = y_1 + np.round(u_1)

      x_1[x_1>W-1] = 0
      y_1[y_1>H-1] = 0

      for i in range(1, int(total_comparisons/one_cycle)):
        #j= i*fincrement + first_frame
        #with open(str(j) + ".pkl", 'rb') as fp:
            #movement = pickle.load(fp)

        #os.remove(str(j) + ".pkl")

        movement = movement_all[i]
        u = movement[2]
        v = movement[3]
        u_1 = getData(y_1.astype('int32'), x_1.astype('int32'), u)
        v_1 = getData(y_1.astype('int32'), x_1.astype('int32'), v)
        posn_1 = getData(y_1.astype('int32'), x_1.astype('int32'), movement[1])

        temp1 = (posn ==i*100).astype(int)

        posn_1 = np.multiply(posn_1, temp1)
        u_1 = np.multiply(u_1, temp1)
        v_1 = np.multiply(v_1, temp1)

        posn = posn + posn_1
        x_1 = x_1 + np.round(v_1)
        y_1 = y_1 + np.round(u_1)
        x_1[x_1>W-1] = 0
        y_1[y_1>H-1] = 0

      # Get last pixels and frame
      x_last = getData(y_1.astype('int32'), x_1.astype('int32'), x_corr)
      y_last = getData(y_1.astype('int32'), x_1.astype('int32'), y_corr)

      frame_last = posn*fincrement/100

      # Calculate speed of each point
      u = get_speed(x_corr, x_last, 0, frame_last)
      u[np.isnan(u)] = 0

      # Calculate height of each pixel
      height = (inputs['GPS_MSL_Alt'][ind] - f*inputs['Grnd_Spd'][ind]/u)/math.cos(math.pi*inputs['Pitch'][ind]/180)
      height[height == -inf] = 0

      # Create output
      x = np.arange(H)
      y = np.arange(W)

      x_coord, y_coord = np.meshgrid(y,x)

      #Output = np.column_stack((y_coord[center_region].ravel(),x_coord[center_region].ravel(), height[center_region].ravel()))
      Output = np.column_stack((y_coord.ravel(),x_coord.ravel(), height.ravel()))

      np.savetxt("/content/drive/MyDrive/Outputs/Output_%d.csv" % ind, Output, delimiter=",", fmt="%.2f",
           header="X,Y,Height", comments="")

      print(f"Time Taken in s: {time.time() - st}", ind)

100%|██████████| 431/431 [03:34<00:00,  2.01it/s]

Time Taken in s: 214.3976788520813 2
